In [1]:
import pyodbc
import pandas as pd

In [9]:
class FaseToRuta:
    def __init__(self):
        self.server_solmicro = r'srvsql'
        self.server_industry = r'SERVIDOR'
        self.database_solmicro = 'SolmicroERP6_PruebasSub'
        self.database_industry = 'IPFavram'
        self.username_solmicro = 'sa'
        self.password_solmicro = 'Altai2021'
        self.password_industry = '71zl6p9h'
        self.connection_string_solmicro = f'DRIVER={{SQL Server}};SERVER={self.server_solmicro};DATABASE={self.database_solmicro};UID={self.username_solmicro};PWD={self.password_solmicro}'
        self.connection_string_industry = f'DRIVER={{SQL Server}};SERVER={self.server_industry};DATABASE={self.database_industry};UID={self.username_solmicro};PWD={self.password_industry}'
        self.connection = None
        self.connection_industry = None
        self.tables_info = []
        self.listArticulos = [443000021633,442295636741,443000021629,443000022021,443000022026,443000022027,443000021525,443000021559,443000021561,443000021562,443000021563,443000021565,443000022076,442296340551,443000021582,443000021583,443000021584,443000021585,443000021627,443000022005,443000021626,443000021625,443000016122,443000021628]

    def Open_Conn_Solmicro(self):
        try:
            self.connection = pyodbc.connect(self.connection_string_solmicro)
        except Exception as e:
            print("Error opening connection: ", e)

    def Open_Conn_Industry(self):
        try:
            self.connection_industry = pyodbc.connect(
                self.connection_string_industry)
        except Exception as e:
            print("Error opening connection: ", e)

    def __del__(self):
        self.CloseConnection()

    def CloseConnection(self):
        try:
            self.connection = pyodbc.connect(self.connection_string_solmicro)
        except Exception as e:
            print("Error opening connection: ", e)

    def CloseConnectionIndustry(self):
        try:
            self.connection_industry = pyodbc.connect(
                self.connection_string_industry)
        except Exception as e:
            print("Error opening connection: ", e)

    def RunQuery(self, stringQuery):
        cursor = self.connection.cursor()
        registers = cursor.execute(stringQuery).fetchall()
        cursor.close()
        return registers
        # return pd.read_sql(sql=stringQuery,con=self.connection)

    def ExecuteForRunQuery(self, listArticulos):
        result = []
        try:
            cursor = self.connection_industry.cursor()
            for id in listArticulos:
                stringQuey = f"select * from MFase where   (Articulo = N'{id}')"
                currentResult = cursor.execute(stringQuey).fetchall()
                result.append(currentResult)
        except Exception as e:
            print("Error en query: ",e)
        finally:
            if cursor:
                cursor.close()
        return result

    def RunQueryIndustry(self, stringQuery):
        cursor = self.connection_industry.cursor()
        registers = cursor.execute(stringQuery).fetchall()
        cursor.close()
        return registers

    @staticmethod
    def get_arti_faltantes(resultArtil, result_art_provee):
        # Crear un diccionario con IDArticulo como clave y DescArticulo como valor
        artil_dict = {item[0]: item[1] for item in resultArtil}
        # Obtener solo los IDArticulo de result_art_provee
        art_provee_ids = set(item[0] for item in result_art_provee)
        # Encontrar los IDArticulo faltantes y mantener el DescArticulo correspondiente
        missing_items = [(artil_id, artil_dict[artil_id])
                         for artil_id in artil_dict if artil_id not in art_provee_ids]
        return missing_items

    @staticmethod
    def get_final_art(missing_items, result_art_provee_industry):
        artil_dict = {item[0]: item[1:] for item in result_art_provee_industry}
        art_provee_ids = set(item[0] for item in missing_items)
        result_items = [(artil_id, *artil_dict[artil_id])
                        for artil_id in artil_dict if artil_id in art_provee_ids]
        return result_items

    @staticmethod
    def missing_providers(proveedores, data):
        set_data_proveedores = set(item[1] for item in data if item[1])
        set_provee = set(item[0] for item in proveedores)
        provee_dict = {item[0]: item[1] for item in proveedores}
        result_missing_proveedores = [
            (provee) for provee in set_data_proveedores if provee and provee not in set_provee]
        return result_missing_proveedores

    @staticmethod
    def export_to_excel(data):
        df = pd.DataFrame(
            data, columns=["IDArticulo", "Proveedor", "Plazo", "Precio"])
        df.to_excel("articulos_sin_proveedor.xlsx", index=False)

In [10]:
iFaseToRuta = FaseToRuta()
iFaseToRuta.Open_Conn_Industry()
# iFaseToRuta.Open_Conn_Solmicro()

In [11]:
iFaseToRuta.ExecuteForRunQuery(iFaseToRuta.listArticulos)

[[('443000021633', 1, 'PROGRAMACION: METER CAJA EMBALAJE + INDICAR MAXIMO PIEZAS CAJA Y PALET S/ LISTA', 'I', '120', 9999.0, 1.0, 0.0, 0.0, None, None, 0.0, 0.0, datetime.datetime(2023, 11, 24, 15, 0, 45), datetime.datetime(2023, 11, 24, 15, 0, 45), None, None, None, 0, 'N', 9999.0, 1.0, 'HOR', 31170, 0.00010001000100010001, 'HOR', None, 0, 0.0),
  ('443000021633', 10, 'PUNZONADO + AVELL+POSICIÓN VERTICAL+SIN ARAÑAZOS(CRONO ALBERTO 60UDS HORA+( 22 MIN CHAPA)', 'C', '130', 30.37, 1.0, 0.0, 0.0, None, None, 0.0, 0.0, datetime.datetime(2022, 3, 4, 12, 29, 33, 270000), datetime.datetime(2023, 11, 15, 0, 0), None, 'cristina', 'MFASE2007S', 3, 'N', 30.37, 1.0, 'H', 214741, 0.03292723081988805, 'HOR', None, 0, 21.28),
  ('443000021633', 15, 'MEC. TALADROS Ø2,75+SIN ARAÑAZOS+POSICÓN VERTICAL', 'C', '170', 28.83, 1.0, 0.0, 0.0, None, None, 0.0, 0.0, datetime.datetime(2022, 3, 4, 12, 29, 33, 437000), datetime.datetime(2023, 11, 15, 0, 0), None, 'cristina', 'MFASE2007S', 3, 'N', 28.83, 1.0, 'H', 